In [21]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [22]:
import multiprocessing
from multiprocessing import Pool
from bb_binary import FrameContainer, Repository, load_frame_container
import pandas as pd
import numpy as np
from collections import namedtuple 
import networkx as nx

from pandas import DataFrame, Series

import bib.preprocessing as prep

# For a given folder, create some networks with a window size
(2^x frames < 1024)

In [23]:
#path = "00_Data/testset_2015_20m/2015082215/"
path = "../data/1h/2015082215/"

In [24]:
ilen = 3
distance = 160
confidence = .95
window_size = 2**8 #256 # 4 netzwerke pro datei #~85 Sekunden

In [25]:
# packt immer die 4 Kameradateien zusammen
def get_files(path):
    repo = Repository(path)
    file = list(repo.iter_fnames())
    a = [f.split('/')[-1].split("_")[1] for f in file]
    l = len(a)/4
    npa = np.array(file).reshape(int(l),4)
    return npa

In [26]:
files = get_files(path)

In [27]:
number_cpu = 4
# multiprocessing.cpu_count()

In [28]:
pool = multiprocessing.Pool( number_cpu )

In [29]:
tasks = []

for e, f in enumerate(files[:2]):
    tasks.append((e, f, confidence, distance, ilen, window_size))

In [30]:
len(tasks)

2

In [31]:
tasks

[(0,
  array([ '/home/mi/aschle/workspace/data/1h/2015082215/2015/08/22/15/00/Cam_3_2015-08-22T15:04:01.415537Z--2015-08-22T15:09:42.430042Z.bbb',
         '/home/mi/aschle/workspace/data/1h/2015082215/2015/08/22/15/00/Cam_0_2015-08-22T15:04:02.986627Z--2015-08-22T15:09:44.012132Z.bbb',
         '/home/mi/aschle/workspace/data/1h/2015082215/2015/08/22/15/00/Cam_2_2015-08-22T15:04:03.113634Z--2015-08-22T15:09:44.128139Z.bbb',
         '/home/mi/aschle/workspace/data/1h/2015082215/2015/08/22/15/00/Cam_1_2015-08-22T15:04:26.298959Z--2015-08-22T15:10:07.314465Z.bbb'], 
        dtype='<U128'),
  0.95,
  160,
  3,
  256),
 (1,
  array([ '/home/mi/aschle/workspace/data/1h/2015082215/2015/08/22/15/00/Cam_3_2015-08-22T15:09:42.763061Z--2015-08-22T15:15:23.777565Z.bbb',
         '/home/mi/aschle/workspace/data/1h/2015082215/2015/08/22/15/00/Cam_0_2015-08-22T15:09:44.345151Z--2015-08-22T15:15:25.360656Z.bbb',
         '/home/mi/aschle/workspace/data/1h/2015082215/2015/08/22/15/00/Cam_2_2015-08-22

In [32]:
def generate_networks(index, file_list, confidence=.95, distance=160, ilen=3, window_size=256):
    print("process {}".format(index))
    
    xmax = 3000
    
    # list of networks
    network_list = []
    
    # one df per cam
    dataframes = np.empty(4, dtype=object)

    for i in list(range(4)):
        fc = load_frame_container(file_list[i])
        df = prep.get_dataframe2(fc)
        df = prep.calcIds(df,confidence)

        camIdx = int(file_list[i].split("/")[-1].split("_")[1])
        dataframes[camIdx] = df

    # cam 0 und cam1 nach rechts verschieben
    dataframes[0].xpos = dataframes[0].xpos + xmax
    dataframes[1].xpos = dataframes[1].xpos + xmax

    # Seiten zusammenfugen
    side0 = pd.concat([dataframes[3], dataframes[0]])
    side1 = pd.concat([dataframes[2], dataframes[1]])

    close1 = prep.get_close_bees_ckd(side0, distance)
    close2 = prep.get_close_bees_ckd(side1, distance)

    close = pd.concat([close1,close2])

    p = prep.bee_pairs_to_timeseries(close)
    
    for w in list(range(1024/window_size)): 
        part = p.ix[:,window_size*w:window_size*(w+1)]
        edges = prep.extract_interactions(part,ilen)
        g = prep.create_graph2(edges)
        network_list.append(((index*1024)+(w*window_size),g))
    
    return network_list

In [33]:
networks = []

In [34]:
results = [pool.apply_async( generate_networks, t, callback=networks.extend ) for t in tasks]

In [35]:
len(networks)

0